<a href="https://colab.research.google.com/github/prizmalty/flower_auction/blob/main/%E8%8A%B1%E3%81%8D_%E3%83%87%E3%83%BC%E3%82%BF%E5%8F%AF%E8%A6%96%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **花き データ可視化**


## pandasのread_csvで読み込む

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dir_path ="/content/drive/MyDrive/python/kaki/Kak_K0.csv"

In [ ]:
df = pd.read_csv(dir_path,encoding="shift_jis")
df

,販売結果（花き 全市場）,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,令和 4年08月30日(火曜日),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,切枝（単位：本）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,品名,卸売数量計,販売方法,卸売数量,北足立,大田,板橋,葛西,世田谷
4,その他の切枝,40,相対,40,-,-,40,-,-
...,...,...,...,...,...,...,...,...,...
74,NaN,NaN,商物分離,10299,-,10299,-,-,-
75,小計,15382,せり・入札,2013,136,1086,764,16,11
76,NaN,NaN,相対,2793,386,2386,-,5,16
77,NaN,NaN,第三者販売,288,-,196,-,92,-


素のpandasで読み込む方法は難しそう

##スクレイピングで取得する

In [15]:
#モジュールインポート
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime 
import pandas as pd
import re

In [16]:
#URL取得
URL="https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202208/20220831/Kak/SN_Kak_Zen_index.html"
response=requests.get(URL)
response

<Response [200]>

In [17]:
#URL取得
URL="https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202208/20220831/Kak/SN_Kak_Zen_index.html"

In [18]:
response=requests.get(URL)
response

<Response [200]>

In [19]:
!pip install chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
soup = BeautifulSoup(response.content, 'html.parser')

In [21]:
table=soup.find_all("div",class_="kekka-page-kak-zen")[2]

In [22]:
dataframe_columns = {
    "日付":"",
    "分類":"",
    "品名":"",
    "販売方法":"",
    "卸売数量":"",
    "北足立":"",
    "大田":"",
    "板橋":"",
    "葛西":"",
    "世田谷":""
}
df = pd.DataFrame([],columns= dataframe_columns.keys())

In [23]:
#ターゲットとなるデータフレームの枠が作成された
df

,日付,分類,品名,販売方法,卸売数量,北足立,大田,板橋,葛西,世田谷


In [24]:
#日付 
#曜日、()、空白など邪魔な文字列を削除する
datetime = soup.find("th",class_ ="header-title-date").text
datetime_fixed = re.sub("[月火水木金土日]曜日","",datetime).replace("()","").replace(" ","")
datetime_fixed

'令和4年08月31日'

In [25]:
import unicodedata
import re
import datetime
 
# 各年号の元年を定義
eraDict = {
    "明治": 1868,
    "大正": 1912,
    "昭和": 1926,
    "平成": 1989,
    "令和": 2019,
}
 
 
def japanese_calendar_converter(text):
    # 正規化
    normalized_text = unicodedata.normalize("NFKC", text)
 
    # 年月日を抽出
    pattern = r"(?P<era>{eraList})(?P<year>[0-9]{{1,2}}|元)年(?P<month>[0-9]{{1,2}})月(?P<day>[0-9]{{1,2}})日".format(eraList="|".join(eraDict.keys()))
    date = re.search(pattern, normalized_text)
 
    # 抽出できなかったら終わり
    if date is None:
        print("Cannot convert to western year")
 
    # 年を変換
    for era, startYear in eraDict.items():
        if date.group("era") == era:
            if date.group("year") == "元":
                year = eraDict[era]
            else:
                year = eraDict[era] + int(date.group("year")) - 1
    
    # date型に変換して返す
    return datetime.date(year, int(date.group("month")), int(date.group("day")))

In [26]:
datetime_fixed

'令和4年08月31日'

In [27]:
datetime_converted = japanese_calendar_converter(datetime_fixed)
print(datetime_converted)

2022-08-31


In [28]:
#分類 テーブルのキャプションから取るのが良いかも 
classification = table.find("caption",class_ ="kekka-page-kak-caption").text
classification 

'切葉（単位：枚）'

In [29]:
#品名 取れた
flower_name = table.find("td",class_="kekka-page-kak-hinmei-zen").text
flower_name

'そてつ'

In [30]:
#販売方法
sales_method =  table.find("td",class_="kekka-page-kak-hanbai-zen").text
sales_method

'せり・入札'

In [31]:
#卸売数量
wholesale_quantity = table.find_all("td",class_="kekka-page-kak-num-zen")[0].text
if wholesale_quantity=="−":
  wholesale_quantity = 0
elif "," in wholesale_quantity:
  wholesale_quantity = int(wholesale_quantity.replace(",",""))
else:
  wholesale_quantity = int(wholesale_quantity)
wholesale_quantity

800

In [32]:
#北足立
north_adachi_quantity = table.find_all("td",class_="kekka-page-kak-num-zen")[1].text
if north_adachi_quantity == "−":
  north_adachi_quantity = 0 
elif "," in north_adachi_quantity:
  north_adachi_quantity = int(north_adachi_quantity.replace(",",""))
else:
  north_adachi_quantity = int(north_adachi_quantity)
north_adachi_quantity

100

In [33]:
#大田
ota_quantity  = table.find_all("td",class_="kekka-page-kak-num-zen")[2].text
if ota_quantity == "−":
  ota_quantity = 0
elif "," in ota_quantity:
  ota_quantity = int(ota_quantity.replace(",",""))
else:
  ota_quantity = int(ota_quantity)
ota_quantity

500

In [34]:
#板橋
#値がない場合、"-"と返ってくる 
#この場合、整数0を返すような仕組みが必要（いつ何時この値が0になるかわからないので）
itabashi_quantity = table.find_all("td",class_="kekka-page-kak-num-zen")[3].text 
if itabashi_quantity == "−":
  itabashi_quantity = 0
elif "," in itabashi_quantity:
  itabashi_quantity = int(itabashi_quantity.replace(",",""))
else:
  itabashi_quantity =int(itabashi_quantity)
itabashi_quantity

0

In [35]:
#葛西
kasai_quantity  = table.find_all("td",class_="kekka-page-kak-num-zen")[4].text
if kasai_quantity == "−":
  kasai_quantity = 0
elif "," in kasai_quantity:
  kasai_quantity = int(kasai_quantity.replace(",",""))
else:
  kasai_quantity = int(kasai_quantity)
kasai_quantity 

200

In [36]:
#世田谷
setagaya_quantity  = table.find_all("td",class_="kekka-page-kak-num-zen")[5].text
if setagaya_quantity == "−":
  setagaya_quantity = 0
elif "," in setagaya_quantity:
  setagaya_quantity = int(setagaya_quantity.replace(",",""))
else:
  setagaya_quantity = int(setagaya_quantity) 
setagaya_quantity 

0

In [37]:
dataframe_value = {
    "日付":datetime_converted,
    "分類":classification,
    "品名":flower_name,
    "販売方法":sales_method,
    "卸売数量":wholesale_quantity,
    "北足立":north_adachi_quantity,
    "大田":ota_quantity,
    "板橋":itabashi_quantity,
    "葛西":kasai_quantity,
    "世田谷":setagaya_quantity
}
data_list=[]
data_list.append(dataframe_value)
df = pd.DataFrame(data_list)
df

,日付,分類,品名,販売方法,卸売数量,北足立,大田,板橋,葛西,世田谷
0,2022-08-31,切葉（単位：枚）,そてつ,せり・入札,800,100,500,0,200,0


# URLを取得する

In [38]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime 
import pandas as pd
import re
#URL取得
URL="https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202208/20220831/Kak/SN_Kak_Zen_index.html"
response=requests.get(URL)
response

<Response [200]>

In [44]:
import datetime
now = datetime.datetime.now()
yyyymm = now.strftime("%Y%m")
yyyymmdd = now.strftime("%Y%m%d")
url = f"https://www.shijou-nippo.metro.tokyo.lg.jp/SN/{yyyymm}/{yyyymmdd}/Kak/SN_Kak_Zen_index.html"
url

'https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202209/20220905/Kak/SN_Kak_Zen_index.html'

In [43]:
import datetime
now = datetime.datetime.now()
yyyymm = now.strftime("%Y%m")
yyyymmdd = now.strftime("%Y%m%d")

In [45]:
yyyymm

'202209'

In [46]:
yyyymmdd

'20220905'

In [54]:
date_tuple = [
    ("202209","20220905"),
    ("202209","20220906")
    ]
date_tuple

[('202209', '20220905'), ('202209', '20220906')]

In [55]:
for i,j in date_tuple:
  url = f"https://www.shijou-nippo.metro.tokyo.lg.jp/SN/{i}/{j}/Kak/SN_Kak_Zen_index.html"
  print(url)

https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202209/20220905/Kak/SN_Kak_Zen_index.html
https://www.shijou-nippo.metro.tokyo.lg.jp/SN/202209/20220906/Kak/SN_Kak_Zen_index.html


In [50]:
from datetime import datetime, timedelta
yesterday = datetime.datetime.now()- timedelta(1)
yesterday_yyyymm = yesterday.strftime("%Y%m")
yesterday_yyyymm

AttributeError: ignored

In [51]:
from datetime import datetime, timedelta

nowadays = datetime.now()

yesterday = nowadays - timedelta(1)
yesterday

datetime.datetime(2022, 9, 4, 23, 35, 19, 803938)

In [52]:
import calendar

print(calendar.month(2019, 1))

    January 2019
Mo Tu We Th Fr Sa Su
    1  2  3  4  5  6
 7  8  9 10 11 12 13
14 15 16 17 18 19 20
21 22 23 24 25 26 27
28 29 30 31

